In [1]:
#include <iostream>

/*a workaround to solve cling issue*/
#include "../inc/macos_cling_workaround.hpp"
/*set libtorch path, load libs*/
#include "../inc/load_libtorch.hpp"
/*import custom defined macros*/
#include "../inc/custom_def.hpp"
/*import matplotlibcpp*/
#include "../inc/load_matplotlibcpp.hpp"
/*import opencv*/
#include "../inc/load_opencv.hpp"

/*import libtorch header file*/
#include <torch/torch.h>
#include <opencv2/opencv.hpp>
#include <cmath>

// Use (void) to silent unused warnings.
#define assertm(exp, msg) assert(((void)msg, exp))

In [2]:
#define VAR_NAME(Variable) (#Variable)

void print_tensor_size(std::string name, torch::Tensor t)
{
    int dims = t.dim();
    std::cout << name << " dims is (";
    for (int i = 0; i < dims; i++) {
        std::cout << t.size(i);
        if (i < (dims - 1)) std::cout << " x ";
    }
    std::cout << ")" << std::endl;
}

# 填充和步幅

**在所有侧边填充1个像素**

In [3]:
// def comp_conv2d(conv2d, X):
//     X = X.reshape((1, 1) + X.shape)
//     Y = conv2d(X)
//     return Y.reshape(Y.shape[2:])

// conv2d = nn.Conv2d(1, 1, kernel_size=3, padding=1)
// X = torch.rand(size=(8, 8))
// comp_conv2d(conv2d, X).shape

In [4]:
torch::Tensor comp_conv2d(torch::nn::Sequential conv2d, torch::Tensor X)
{
//     auto s = X.sizes(); //c10::IntArrayRef
//     X = X.reshape({s[0]+1, s[1]+1});
    X = torch::constant_pad_nd(X, at::IntArrayRef({0,1}));
    printT(X.sizes());
    auto Y = conv2d->forward(X);
    printT(Y.sizes());
    return Y.reshape(at::IntArrayRef({Y.size(2), Y.size(3)}));
}

In [5]:
torch::nn::Sequential conv2d{torch::nn::Conv2d(torch::nn::Conv2dOptions(1, 1, {3, 3}).stride(1).padding(1).bias(false))};
auto X = torch::rand({8, 8});
auto Z = comp_conv2d(conv2d, X);
printT(Z.sizes());

X.sizes() = 
[8, 9]
<<--->>



Standard Exception: Expected 4-dimensional input for 4-dimensional weight [1, 1, 3, 3], but got 2-dimensional input of size [8, 9] instead
Exception raised from check_shape_forward at /home/lh/github/libtorch/pytorch/aten/src/ATen/native/Convolution.cpp:454 (most recent call first):
frame #0: c10::Error::Error(c10::SourceLocation, std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> >) + 0x69 (0x7fef789c1fd9 in /opt/libtorch/lib/libc10.so)
frame #1: at::native::_convolution(at::Tensor const&, at::Tensor const&, at::Tensor const&, c10::ArrayRef<long>, c10::ArrayRef<long>, c10::ArrayRef<long>, bool, c10::ArrayRef<long>, long, bool, bool, bool, bool) + 0x1fad (0x7fef6583f2ed in /opt/libtorch/lib/libtorch_cpu.so)
frame #2: <unknown function> + 0x12f3087 (0x7fef660ad087 in /opt/libtorch/lib/libtorch_cpu.so)
frame #3: <unknown function> + 0x1354222 (0x7fef6610e222 in /opt/libtorch/lib/libtorch_cpu.so)
frame #4: <unknown function> + 0x8db1d5 (0x7fef656951d5 in /opt/li